# Expand the incident wave using sphercial wave functions

In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math
import matplotlib.pyplot as plt
bempp.core.opencl_kernels.set_default_device(0,0)

### Analytical expansion of $u^{i}(x) = e^{ikr\hat{x}\cdot\hat{y}} = \sum_{q = 0}^{\infty}\sum_{p=-q}^{q}a_{q}^{p}j_{q}(kr)Y_{q}^{p}(\hat{x})$

In [9]:
h = 0.3 # Size of the mesh
d = [1,0,0] # wave's travel direction
wavenumber = 5 # wavenumber

In [16]:
unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_sphere = unit_sphere.vertices
space_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

dom_sphere = bempp.api.shapes.sphere(r = 0.9, origin = [0,0,0], h = h)
space_dom_sphere = bempp.api.function_space(dom_sphere,'P',1)

### Numerical expansion over sphere surrounding our domain

### $Y_{q}^{p}$, $j_{q}(kr)$, $\tilde{e_{q}^{p}}(x) = j_{q}(kr)Y_{q}^{p}(\hat{x})$

In [4]:
def normalized_spherical_harmonics(p, q, x):
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    legd_poly = np.array(scipy.special.lpmn(abs(p),q, np.cos(polar))[0]) [-1,-1] 
    return (-1)**((p + abs(p))/2)*np.sqrt(((2*q+1)/(4*np.pi))*(math.factorial(q-abs(p))/math.factorial(q+abs(p))))*legd_poly*np.exp(1j*p*azimuth)

In [5]:
def normalized_spherical_harmonics(p, q, x):
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)
def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [4]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*wavenumber*r))*scipy.special.jv(q+0.5, wavenumber*r)


def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

### $a_{q}^{p} = 4\pi(i)^{q}Y_{q}^{-p}(\hat{y})/(-1)^{|p|}$

In [20]:
def coeff_sph_expansion(p,q):
    """Analytical formula of the coefficients of the spherical expansions of plane waves"""
    coeff = 4*np.pi*((1j)**q)*normalized_spherical_harmonics(-p,q,d)/(-1)**(abs(p))
    coeff = 4*np.pi*((1j)**q)*np.conj(normalized_spherical_harmonics(p,q,d))*(-1)**(- p - abs(p))
    return coeff

In [17]:
@bempp.api.complex_callable
def inc_wave_fun(x,n,domain_index, result):
    result[0] = np.exp(1j*k*x[0])
    
inc_wave_grid_fun_sphere = bempp.api.GridFunction(space_dom_sphere, fun = inc_wave_fun)

In [21]:
deg = 10
@bempp.api.complex_callable
def wave_expansion_sphere(x,n,domain_index,result):
    with objmode():
        basis_fun = []    
        coeff = []
        for q in range(deg+1):
            for p in range(-q,q+1):
                basis_fun.append(regular_spherical_wavefunctions(p,q,x))
                coeff.append(coeff_sph_expansion(p,q))
        result[0] = np.inner(coeff,basis_fun)

wave_grid_fun_sphere = bempp.api.GridFunction(space_dom_sphere, fun = wave_expansion_sphere)
error_sphere = (inc_wave_grid_fun_sphere - wave_grid_fun_sphere).l2_norm()

In [22]:
error_sphere

0.0003733108481986568